In [5]:
import sys
import time
import pandas as pd

boiler_plate = f"""
/*
Created      : {time.strftime("%Y-%m-%d %H:%M")}
Generated_by : Python
*/

"""

def create_data_type(row):
    s = row[0]
    if "%p1" in s:
        s = s.replace("%p1",str(int(row[1])))
    if "%p2" in s:
        s = s.replace("%p2",str(int(row[2])))
    return s

def create_sql_string(entity_name,column_name,index,data_types,nullable):
    
    columns_str = []
    for name,data_type,nul in zip(column_name,data_types,nullable):
        columns_str.append(f"[{name}] {data_type} {'NOT NULL' if nul == 'N' else 'NULL'}")
    columns_str = ",\n ".join(columns_str)
    
    str = f"""CREATE TABLE [IF NOT EXISTS] [{entity_name}]
(
 {columns_str}
);
    
"""

    return str

## Read Data
df = pd.read_excel("/content/FACT_PAYMENT_postgres.xlsx") ## File name


In [3]:
df["New Data types"] = df[['Target data type', 'Length', 'Precision']].apply(create_data_type,axis=1)
df.drop(['Target data type', 'Length', 'Precision'],axis=1,inplace=True)
grouped_df = df.groupby("Target Table Name").agg(list)


In [10]:

## Saving the SQL Data
with open("output.SQL", "w") as f:
    f.write(boiler_plate)
    for index, row in grouped_df.iterrows():
        sql_command = create_sql_string(index,\
                                        row["Target column name"],\
                                        row["Index"][0],\
                                        row["New Data types"],\
                                        row["Nullable"])
        f.write(f"-- Table {index} \n\n")
        f.write(sql_command)